In [ ]:
# Load the CSV file to inspect its structure and contents
import pandas as pd

# Load the file
file_path = 'Dev_data_to_be_shared.csv'
df = pd.read_csv(file_path, encoding='ascii')

# Display the first few rows and column information
print(df.head())
print(df.info())

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load and prepare data
df = pd.read_csv('Dev_data_to_be_shared.csv')

# Basic statistics and missing values
print("Dataset Shape:", df.shape)
print("\nMissing Values:\n", df.isnull().sum().sum())

# Calculate correlation with bad_flag
correlations = df.corr()['bad_flag'].sort_values(ascending=False)
print("\nTop 10 Features Correlated with bad_flag:\n", correlations[:10])

# Create some basic feature engineering
# 1. Aggregate statistics for transaction attributes
df['trans_mean'] = df.filter(like='transaction_attribute').mean(axis=1)
df['trans_std'] = df.filter(like='transaction_attribute').std(axis=1)
df['trans_max'] = df.filter(like='transaction_attribute').max(axis=1)
df['trans_min'] = df.filter(like='transaction_attribute').min(axis=1)

# Plot distribution of bad_flag
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='bad_flag')
plt.title('Distribution of Bad Flag')
plt.show()

# Plot correlation heatmap of new features
new_features = ['trans_mean', 'trans_std', 'trans_max', 'trans_min', 'bad_flag']
plt.figure(figsize=(10, 8))
sns.heatmap(df[new_features].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation of Engineered Features')
plt.show()

# Calculate feature importance based on correlation with bad_flag
feature_importance = abs(df[new_features].corr()['bad_flag']).sort_values(ascending=False)
print("\nEngineered Feature Importance:\n", feature_importance)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('Dev_data_to_be_shared.csv')

# Count zeros in each transaction attribute
zero_counts = df.filter(like='transaction_attribute').apply(lambda x: (x == 0).sum())
top_zero_cols = zero_counts.sort_values(ascending=False).head(10)

print("Top 10 Transaction Attributes with Most Zeros:")
print(top_zero_cols)

# Analyze patterns when specific attributes are 0
# Find interesting correlations when certain attributes are 0
interesting_patterns = []
transaction_cols = df.filter(like='transaction_attribute').columns

for col in transaction_cols[:10]:  # Check first 10 attributes
    zero_mask = df[col] == 0
    non_zero_mask = df[col] != 0
    
    if zero_mask.sum() > 0 and non_zero_mask.sum() > 0:
        zero_bad_rate = df[zero_mask]['bad_flag'].mean()
        non_zero_bad_rate = df[non_zero_mask]['bad_flag'].mean()
        
        if abs(zero_bad_rate - non_zero_bad_rate) > 0.05:  # Significant difference
            interesting_patterns.append({
                'attribute': col,
                'zero_bad_rate': zero_bad_rate,
                'non_zero_bad_rate': non_zero_bad_rate,
                'difference': abs(zero_bad_rate - non_zero_bad_rate)
            })

print("\nInteresting Patterns Found:")
for pattern in sorted(interesting_patterns, key=lambda x: x['difference'], reverse=True):
    print(f"\n{pattern['attribute']}:")
    print(f"Bad rate when 0: {pattern['zero_bad_rate']:.4f}")
    print(f"Bad rate when non-zero: {pattern['non_zero_bad_rate']:.4f}")
    print(f"Difference: {pattern['difference']:.4f}")

# Visualize relationship between number of zeros and bad_flag
df['zero_count'] = df.filter(like='transaction_attribute').apply(lambda x: (x == 0).sum(), axis=1)
plt.figure(figsize=(10, 6))
sns.boxplot(x='bad_flag', y='zero_count', data=df)
plt.title('Number of Zero Transactions vs Bad Flag')
plt.show()

# Check if there's a pattern in consecutive zeros
df['consecutive_zeros'] = df.filter(like='transaction_attribute').apply(
    lambda x: x.rolling(window=3).apply(lambda x: (x == 0).all()).max(), axis=1
)

print("\nBad flag rate for accounts with/without consecutive zeros:")
print(df.groupby('consecutive_zeros')['bad_flag'].mean())

In [ ]:
# Optimizing the consecutive zero analysis to avoid timeout
# Instead of applying rolling window to all columns, we will calculate consecutive zeros row-wise

def calculate_consecutive_zeros(row):
    max_consecutive = 0
    current_consecutive = 0
    for value in row:
        if value == 0:
            current_consecutive += 1
            max_consecutive = max(max_consecutive, current_consecutive)
        else:
            current_consecutive = 0
    return max_consecutive

# Apply the optimized function to calculate consecutive zeros
df['consecutive_zeros'] = df.filter(like='transaction_attribute').apply(calculate_consecutive_zeros, axis=1)

# Analyze bad_flag rate for accounts with/without consecutive zeros
print("\nBad flag rate for accounts with/without consecutive zeros:")
print(df.groupby('consecutive_zeros')['bad_flag'].mean())

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('Dev_data_to_be_shared.csv')

# Calculate percentage of zeros for each transaction attribute
zero_percentages = (df.filter(like='transaction_attribute') == 0).mean() * 100
high_zero_cols = zero_percentages[zero_percentages > 70].sort_values(ascending=False)

print("Transaction attributes with >70% zeros:")
print(high_zero_cols)

# Analyze bad_flag rates for specific patterns
print("\nBad flag rates for different scenarios:")
print("Overall bad rate:", df['bad_flag'].mean())

# Check if having all zeros in first 10 transactions is significant
first_10_zeros = (df.filter(like='transaction_attribute').iloc[:, :10] == 0).all(axis=1)
print("Bad rate when first 10 transactions are all zero:", df[first_10_zeros]['bad_flag'].mean())
print("Bad rate when first 10 transactions are not all zero:", df[~first_10_zeros]['bad_flag'].mean())

# Correlation between number of zero transactions and bad_flag
df['zero_count'] = (df.filter(like='transaction_attribute') == 0).sum(axis=1)
correlation = df['zero_count'].corr(df['bad_flag'])
print("\nCorrelation between number of zero transactions and bad_flag:", correlation)

# Plot distribution of zero counts by bad_flag
plt.figure(figsize=(10, 6))
sns.boxplot(x='bad_flag', y='zero_count', data=df)
plt.title('Distribution of Zero Transaction Counts by Bad Flag')
plt.show()

In [ ]:
# Exploring patterns in non-zero transaction values and other potential insights

# Filter non-zero transaction values for analysis
non_zero_values = df.filter(like='transaction_attribute').replace(0, np.nan)

# Calculate basic statistics for non-zero values
non_zero_stats = non_zero_values.describe()
print("Statistics for non-zero transaction values:")
print(non_zero_stats)

# Check correlation of non-zero transaction attributes with bad_flag
correlations = df.filter(like='transaction_attribute').corrwith(df['bad_flag'])
high_corr = correlations[correlations.abs() > 0.05].sort_values(ascending=False)
print("\nTransaction attributes with highest correlation to bad_flag:")
print(high_corr)

# Visualize distribution of a few highly correlated attributes
high_corr_cols = high_corr.index[:3]
plt.figure(figsize=(12, 6))
for i, col in enumerate(high_corr_cols):
    plt.subplot(1, 3, i + 1)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# Let's analyze patterns in transaction sequences and value distributions

# Calculate transaction value changes
trans_cols = [col for col in df.columns if 'transaction_attribute' in col]
df['value_changes'] = df[trans_cols].diff(axis=1).abs().mean(axis=1)

# Identify accounts with unusual transaction patterns
df['transaction_std'] = df[trans_cols].std(axis=1)
df['transaction_mean'] = df[trans_cols].mean(axis=1)
df['coefficient_variation'] = df['transaction_std'] / df['transaction_mean']

# Analyze bad rates for different transaction patterns
print("Bad rates by transaction variability quartiles:")
df['cv_quartile'] = pd.qcut(df['coefficient_variation'].replace([np.inf, -np.inf], np.nan), 4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
print(df.groupby('cv_quartile')['bad_flag'].mean())

# Visualize relationship between transaction variability and bad rate
plt.figure(figsize=(10, 6))
sns.boxplot(x='cv_quartile', y='value_changes', hue='bad_flag', data=df)
plt.title('Transaction Value Changes by Variability Quartile and Bad Flag')
plt.xticks(rotation=45)
plt.show()

# Find sequential patterns
df['consecutive_increases'] = df[trans_cols].diff(axis=1).gt(0).sum(axis=1)
print("\nBad rates by number of consecutive increases:")
print(df.groupby(pd.qcut(df['consecutive_increases'], 4))['bad_flag'].mean())

In [ ]:
# Adjusting for duplicate bin edges in qcut
# Allow duplicates to handle identical values in 'consecutive_increases'

# Re-run the grouping with duplicates allowed
df['consecutive_increases'] = df[trans_cols].diff(axis=1).gt(0).sum(axis=1)
bad_rates_by_increases = df.groupby(pd.qcut(df['consecutive_increases'], 4, duplicates='drop'))['bad_flag'].mean()

print("Bad rates by number of consecutive increases (adjusted for duplicates):")
print(bad_rates_by_increases)